## Libaries and settings

In [1]:
# Libaries

from flask import Flask, request
from flask_cors import CORS
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
import pickle
import os

app = Flask(__name__)
CORS(app)

### connection to frontend

In [2]:
@app.route("/api/predict/", methods=['GET'])
def predict():
    bfs_number = int(request.args['bfs_number'])
    area = float(request.args['area'])
    rooms = float(request.args['rooms'])
    
    df_bfs_data = pd.read_csv(os.path.join(os.path.dirname(os.path.abspath(__file__)), 'bfs_municipality_and_tax_data.csv'),
                            sep=',', encoding='utf-8')
    df_bfs_data['tax_income'] = df_bfs_data['tax_income'].str.replace("'", "")
    df = df_bfs_data[df_bfs_data['bfs_number']==bfs_number]

    randomforest_model = RandomForestRegressor()
    model_filename = os.path.join(os.path.dirname(os.path.abspath(__file__)), "randomforest_regression.pkl")
    with open(model_filename, 'rb') as f:
        randomforest_model = pickle.load(f)

    # ['rooms' 'area' 'pop' 'pop_dens' 'frg_pct' 'emp' 'tax_income' 'm2_per_rooms']
    prediction = randomforest_model.predict([[rooms, area, df['pop'].iloc[0], df['pop_dens'].iloc[0], df['frg_pct'].iloc[0], df['emp'].iloc[0], df['tax_income'].iloc[0], area/rooms]])
    return str(round(prediction[0],2))

@app.route("/")
def hello_world():

    print(request.args)

    return "<p>Hello, World!</p>"
    

## Import data

In [3]:
# Read and select variables
df_orig = pd.read_csv("bfs_municipality_and_tax_data.csv", sep=",", encoding='utf-8')

# Remove missing values
df = df_orig.dropna()

# Remove duplicates
df = df.drop_duplicates()

# Berechnen des Interquartilbereichs (IQR) für jede Spalte außer bfs_number und bfs_name
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1

# Identifizieren von Ausreißern
outliers = ((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)

# Entfernen von Ausreißern
df_cleaned = df[~outliers]

print(df_cleaned.shape)
df.head(5)

(1719, 7)


C:\Users\Kevin Keo\AppData\Local\Temp\ipykernel_26936\3916130010.py:16: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version. Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  outliers = ((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)


,bfs_number,bfs_name,pop,pop_dens,frg_pct,emp,tax_income
0,1,Aeugst am Albis,1981,250.442478,14.184755,442.0,108'788
1,2,Affoltern am Albis,12303,1161.756374,28.700317,6920.0,72'583
2,3,Bonstetten,5572,749.932705,16.564968,1014.0,91'002
3,4,Hausen am Albis,3751,275.808824,16.022394,1021.0,91'766
4,5,Hedingen,3778,578.560490,16.410799,1478.0,94'456


In [4]:
len(df_cleaned)

1719

In [5]:
df.columns

Index(['bfs_number', 'bfs_name', 'pop', 'pop_dens', 'frg_pct', 'emp',
       'tax_income'],
      dtype='object')

In [6]:
# Create train and test samples
X_train, X_test, y_train, y_test = train_test_split(df[['bfs_number', 'bfs_name', 'pop', 'pop_dens', 'frg_pct',
                                                        'emp', 'tax_income']], 
                                                         df['pop'], 
                                                        test_size=0.20, 
                                                        random_state=42)

In [7]:
# train linear_model = LinearRegression()
linear_model = LinearRegression()

scores = cross_val_score(linear_model, X_train, y_train, scoring="neg_root_mean_squared_error", cv=5)
print('--{}--'.format(linear_model))
print(scores)
print(np.mean(scores)) 

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Kevin Keo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Kevin Keo\anaconda3\lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\Kevin Keo\anaconda3\lib\site-packages\sklearn\linear_model\_base.py", line 578, in fit
    X, y = self._validate_data(
  File "c:\Users\Kevin Keo\anaconda3\lib\site-packages\sklearn\base.py", line 650, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "c:\Users\Kevin Keo\anaconda3\lib\site-pack